In [20]:
import json
from openai import OpenAI

client = OpenAI(
    api_key = None,
)

ImportError: cannot import name 'OpenAI' from 'openai' (/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/openai/__init__.py)

In [22]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 33.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.


In [25]:
openai.__version__

'0.28.0'

In [6]:
# Load the JSON data from the provided file
file_path = '/teamspace/studios/this_studio/ThinkAI_Repo/data/wohnberechtigungsschein_antrag_fields.json'  # Replace with your file path
with open(file_path, 'r') as file:
    form_data = json.load(file)

In [7]:
form_data

[{'title': 'Antragstellerin / Antragsteller',
  'content': [[{'id': 'field_1',
     'label': 'Familienname, ggf. Geburtsname',
     'type': 'iText',
     'width': 45},
    {'id': 'field_2', 'label': 'Vorname(n)', 'type': 'iText', 'width': 30},
    {'id': 'field_3', 'label': 'Geburtsdatum', 'type': 'iDate', 'width': 25}],
   [{'id': 'field_4',
     'label': 'E-Mail-Adresse',
     'type': 'iEmail',
     'width': 70},
    {'id': 'field_5', 'label': 'Telefon', 'type': 'iPhone', 'width': 30}],
   [{'id': 'field_6',
     'label': 'Straße, Hausnummer, Stockwerk, ggf. Wohnungsnummer, Postleitzahl, Ort',
     'type': 'iText',
     'width': 70},
    {'id': 'field_7',
     'label': 'Staatsangehörigkeit',
     'type': 'iText',
     'width': 30}],
   [{'id': 'field_8',
     'label': 'Bei Nicht-EU-Staatsangehörigkeit (auch von Haushaltsmitgliedern) sind Unterlagen über den Aufenthaltsstatus beizufügen.',
     'type': 'text',
     'width': 100}]]},
 {'title': 'Nur bei Antrag auf Erteilung eines Wohnb

In [8]:
# Extract the labels and prepare them for reformulation
labels_to_reformulate = {}
for section in form_data:
    for content_block in section['content']:
        for field in content_block:
            labels_to_reformulate[field['id']] = field['label']

In [9]:
labels_to_reformulate

{'field_1': 'Familienname, ggf. Geburtsname',
 'field_2': 'Vorname(n)',
 'field_3': 'Geburtsdatum',
 'field_4': 'E-Mail-Adresse',
 'field_5': 'Telefon',
 'field_6': 'Straße, Hausnummer, Stockwerk, ggf. Wohnungsnummer, Postleitzahl, Ort',
 'field_7': 'Staatsangehörigkeit',
 'field_8': 'Bei Nicht-EU-Staatsangehörigkeit (auch von Haushaltsmitgliedern) sind Unterlagen über den Aufenthaltsstatus beizufügen.',
 'field_9': 'Ich bewohne zur Zeit eine geförderte Wohnung',
 'field_10': 'Verfügen Sie und / oder zu Ihrem Haushalt gehörende Personen über Vermögen im Wert von mehr als 60.000 EUR?',
 'field_11': "Bei 'ja' machen Sie bitte nachfolgend genauere Angaben"}

In [14]:
# Function to reformulate a label into a question using GPT
def reformulate_question(label):
    prompt = f"Reformulate the following form field label into a clear and simple question: {label}"
    
    # Use the new API call
    response = openai.Completion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=60,
        temperature=0.7
    )



    completion = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = "Say this is a test",
        max_tokens = 7,
        temperature = 0
    )

    question = response['choices'][0]['message']['content'].strip()
    return question

In [15]:
labels_to_reformulate["field_6"]

'Straße, Hausnummer, Stockwerk, ggf. Wohnungsnummer, Postleitzahl, Ort'

In [16]:
# Store the reformulated questions in a dictionary
reformulated_questions = {}

reformulated_questions["field_6"] = reformulate_question(labels_to_reformulate["field_6"])

InvalidRequestError: Unrecognized request argument supplied: messages

In [ ]:
# Store the reformulated questions in a dictionary
reformulated_questions = {}

def reformulate_labels(labels_dict):
    for id, label in labels_dict.items():
        reformulated_questions[id] = reformulate_question(label)


In [4]:
reformulated_questions

{}

In [14]:
! pip install openai==0.28


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:


# Store the reformulated questions in a dictionary
reformulated_questions = {}

def reformulate_labels(labels_dict):
    for id, label in labels_dict.items():
        reformulated_questions[id] = reformulate_question(label)

# Trigger reformulation (e.g., upon user action)
trigger = input("Do you want to reformulate the labels into questions? (yes/no): ").lower()
if trigger == 'yes':
    reformulate_labels(labels_to_reformulate)
    print("Reformulation complete.")
else:
    print("Reformulation not triggered.")

# Output the resulting dictionary
output_file_path = '/mnt/data/reformulated_questions.json'  # Replace with your desired output path
with open(output_file_path, 'w') as output_file:
    json.dump(reformulated_questions, output_file, indent=4)

print(f"Reformulated questions have been saved to {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/wohnberechtigungsschein_antrag_fields.json'